<a href="https://colab.research.google.com/github/SIDIBEMoussa/Muter-automatiquement-bad-words-dans-les-audios-ou-videos/blob/main/muter_bad_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import importlib
import importlib.util
import subprocess
import sys
import os
from pathlib import Path

In [2]:
def ensure_package(package_name,import_name = None):
    try:
        if not import_name:
            import_name = package_name
        importlib.import_module(import_name)
    except ImportError:
        print(f"Installing {package_name}...\n", end=' ', flush=True)
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name, "--quiet"])
        print("Installing done")
    else:
        print("Package already installed")

def clone_repo_quietly(repo_url, target_dir=None):
    target_dir = target_dir or repo_url.split('/')[-1].replace('.git', '')
    if not Path(target_dir).exists():
        subprocess.run(
            ["git", "clone", "--quiet", repo_url, target_dir],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
        )
    print("Cloning done")

In [3]:
ensure_package("git+https://github.com/openai/whisper.git","whisper")
clone_repo_quietly("https://github.com/LDNOOBW/List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words")

Package already installed
Cloning done


In [4]:
def load_bad_words(language="fr"):

    with open(f"List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words/{language}", "r", encoding="utf-8") as f:
        return set(word.strip().lower() for word in f if word.strip())

bad_words_fr = load_bad_words("fr")
bad_words_en = load_bad_words("en")
bad_words_all = bad_words_fr.union(bad_words_en)


In [5]:
import whisper

In [6]:
def transcribe_video(video_path, language="fr"):
    model = whisper.load_model("medium")
    result = model.transcribe(video_path, language=language, word_timestamps=True)
    return result
result = transcribe_video("movie.mp4", "en")

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [17]:
#result['segments'][0]

In [8]:

def get_bad_words(result,language="fr"):
    profanity_timestamps = []
    if language == "fr":
        bad_words = bad_words_fr
    elif language == "en":
        bad_words = bad_words_en
    else:
        bad_words = bad_words_all

    for seg in result["segments"]:
        for word in seg["words"]:
            if word["word"].strip().lower() in bad_words:
                profanity_timestamps.append((word["start"], word["end"]))
    return profanity_timestamps

In [9]:
profanity_timestamps = get_bad_words(result,language="eng")
profanity_timestamps

[]

In [10]:
def mute_profanities(input_video, output_video, profanity_timestamps):
    # Construire le filtre audio
    filters = []
    for start, end in profanity_timestamps:
        filters.append(f"volume=enable='between(t,{start},{end})':volume=0")

    volume_filter = ",".join(filters)

    # Commande ffmpeg
    command = [
        "ffmpeg",
        "-i", input_video,
        "-af", volume_filter,
        "-c:v", "copy",  # on ne touche pas à la vidéo
        "-y",  # overwrite
        output_video
    ]

    print("Running FFmpeg command:")
    print(" ".join(command))

    subprocess.run(command)

In [11]:
mute_profanities(
    input_video="movie.mp4",
    output_video="clean_movie.mp4",
    profanity_timestamps=profanity_timestamps
)

Running FFmpeg command:
ffmpeg -i movie.mp4 -af  -c:v copy -y clean_movie.mp4


In [12]:
def write_srt(transcription_result, srt_filename="captions.srt"):
    def format_timestamp(seconds):
        h = int(seconds // 3600)
        m = int((seconds % 3600) // 60)
        s = int(seconds % 60)
        ms = int((seconds - int(seconds)) * 1000)
        return f"{h:02}:{m:02}:{s:02},{ms:03}"

    with open(srt_filename, "w", encoding="utf-8") as f:
        for i, seg in enumerate(transcription_result["segments"], 1):
            start = format_timestamp(seg["start"])
            end = format_timestamp(seg["end"])
            text = seg["text"].strip()
            f.write(f"{i}\n{start} --> {end}\n{text}\n\n")

write_srt(result)

In [13]:
#! ffmpeg -i movie.mp4 -vf subtitles=captions.srt -c:a copy output_with_subs.mp4

In [14]:
import subprocess

def burn_captions(input_video, output_video, srt_file="captions.srt"):
    cmd = [
        "ffmpeg",
        "-i", input_video,
        "-vf", f"subtitles={srt_file}",
        "-c:a", "copy",
        "-y",  # overwrite
        output_video
    ]
    subprocess.run(cmd)

In [15]:
result = whisper.load_model("medium").transcribe("movie.mp4")

write_srt(result, "captions.srt")
burn_captions("movie.mp4", "movie_captioned.mp4", "captions.srt")


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
